In [58]:
import torch as torch
import torch.nn as nn
import numpy as np
import math

3-layers CNN network

In [94]:
class CNN_beseline(nn.Module):
    def __init__(self):
        super(CNN_beseline, self).__init__()
        self.batch_size = 100
        conv_kernel_size = 8
        pool_kernel_size = 4
        nkernels = [320,480,960]
        
        sequence_length =  1000
        n_genomic_features  = 919
 
        self.cnn_layer1 = nn.Sequential(
            nn.Conv1d(4, nkernels[0], kernel_size = conv_kernel_size),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=pool_kernel_size, stride=pool_kernel_size),
            nn.Dropout(p=0.2)
        )
        self.cnn_layer2 = nn.Sequential(
            nn.Conv1d(nkernels[0], nkernels[1], kernel_size = conv_kernel_size),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=pool_kernel_size, stride=pool_kernel_size),
            nn.Dropout(0.2)
        )
        self.cnn_layer3 = nn.Sequential(    
            nn.Conv1d(nkernels[1], nkernels[2], kernel_size = conv_kernel_size),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5)
            )
        
        
        reduce_by = conv_kernel_size - 1
        pool_kernel_size = float(pool_kernel_size)
        self.n_channels = int(
            np.floor(
                (np.floor(
                    (sequence_length - reduce_by) / pool_kernel_size)
                 - reduce_by) / pool_kernel_size)
            - reduce_by)
        
        self.classifier = nn.Sequential(
            nn.Linear(960 * self.n_channels, n_genomic_features),
            nn.ReLU(inplace=True),
            nn.Linear(n_genomic_features, n_genomic_features),
            nn.Sigmoid())
        
        params = self.parameters()
        self.optimizer = torch.optim.Adam(params,lr = 0.001)

    def forward(self, x):

        out = self.cnn_layer1(x)
        out = self.cnn_layer2(out)
        out = self.cnn_layer3(out)
        #print("out.shape: ",out.shape)
        reshape_out = out.reshape(out.size(0), 960 * self.n_channels)
        predict = self.classifier(reshape_out)
        return predict

  
        return out



Train

In [92]:
def train(model, train_inputs, train_labels):
    loss = nn.BCELoss()
    for i in range(int(train_inputs.shape[0]/model.batch_size)):
        train_x_batch = train_inputs[i*model.batch_size:(i+1)*model.batch_size]
        train_y_batch = train_labels[i*model.batch_size:(i+1)*model.batch_size]
        
        train_x_batch = torch.tensor(train_x_batch)
        train_y_batch = torch.tensor(train_y_batch)
        train_y_batch = train_y_batch.float()
        logits = model.forward(train_x_batch.float())
        l = loss(logits,train_y_batch)
        
        print("loss: ",l)
        model.optimizer.zero_grad()
        l.backward()
        model.optimizer.step()
        

In [65]:
def test(model, test_inputs, test_labels):
    total_loss = 0
    for i in range(int(test_inputs.shape[0]/model.batch_size)):
        test_x_batch = test_inputs[i*model.batch_size:(i+1)*model.batch_size]
        test_y_batch = test_labels[i*model.batch_size:(i+1)*model.batch_size]

        logits = model.call(test_x_batch, True)
        loss = model.loss()
        total_loss += loss
    return total_loss/int(test_inputs.shape[0]/model.batch_size)

Main Function


In [66]:
def main():
    # train label and data
    labels = np.load('./processed_data/testdata_0_10000/reshape_labels_0_10000.npy')
    data = np.load('./processed_data/testdata_0_10000/reshape_data_0_10000.npy')
    
    model = CNN_beseline()
    for i in range(1):
        print("-----------epoch "+str(i+1)+"--------------")
        train(model, data, labels)
   # print('final accuracy {}'.format(test(model, test_inputs, test_labels)))
    
    #test_input_part = test_inputs[:10]
    #test_label_part = test_labels[:10]
    #visualize_results(test_input_part, model.call(test_input_part), test_label_part, CAT, DOG)
    return

In [93]:
main()

-----------epoch 1--------------
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.6925, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.6823, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.3115, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.5960, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.3218, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.2544, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.2520, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.2226, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.1272, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.0781, grad_fn=<Binary

out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.1125, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.1224, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.0719, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.0762, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.0819, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.0595, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.0981, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.0882, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.0781, grad_fn=<BinaryCrossEntropyBackward>)
out.shape:  torch.Size([100, 960, 53])
loss:  tensor(0.0758, grad_fn=<BinaryCrossEntropyBackward>)
out.shape: